In [3]:
%matplotlib notebook
from IPython.display import display
from skimage.io import imread
import ipywidgets as widgets
import matplotlib.pyplot as plt

from python_guis.model import add_node, segment_one_image
from python_guis import INSECTS


def on_canvas_click(event):
    """Add a node and dislay on the image canvas when clicked."""
    if len(nodes) == 0:
        event.inaxes.lines.clear()
        
    add_node(event, nodes, event.canvas)
    
    if len(nodes) >= 3:
        segment_button.disabled = False
    if len(nodes) > 0:
        remove_button.disabled = False
    
def perform_segmentation(*args):
    """Use the current list of nodes to perform a segmentation and redraw the image
    with the initial contour and segmentation result shown."""
    segment, initial = segment_one_image(
        img, nodes, sigma=slider.value, resolution=int(text_field.value), degree=radio.value
    )
    nodes.clear()
    redraw(axes, segment=segment, initial=initial)
    segment_button.disabled = True

def clear_all(*args):
    """Remove all nodes and any displayed segmentation results."""
    nodes.clear()
    remove_button.disabled = True
    segment_button.disabled = True
    axes.lines.clear()
    axes.get_legend().remove()
    fig.canvas.draw()
    
def redraw(axes, segment=None, initial=None):
    """Redraw the image onto axes with lines for an initial contour and/or segmentation
    result if provided."""
    if initial is not None:
        axes.plot(*initial.T, color='blue', label="Initial")

    if segment is not None:
        axes.plot(*segment.T, color='orange', label="Segmented")

    if segment is not None or initial is not None:
        axes.legend()
        
    fig.canvas.draw()

def draw(axes):
    """Plot the image onto axes and add an explanatory title."""
    axes.imshow(img, cmap=plt.get_cmap("binary_r"))
    axes.set_title(
            "Left click to add a control node.\n"
            "At least 3 are needed to perform a segmentation."
    )

img = imread(INSECTS, as_gray=True)
nodes = []
    
# create widgets for left column
slider = widgets.IntSlider(value=1, min=0, max=10, step=1, description="Gaussian filter width:", style={'description_width': 'initial'})
label = widgets.Label(value="Spline parameters")
radio = widgets.RadioButtons(options=[1, 3, 5], description="Degree:")
text_field = widgets.Text(value="360", description="Resolution:")
segment_button = widgets.Button(description="Perform Segmentation", disabled=True, layout=widgets.Layout(width="98%"))
remove_button = widgets.Button(description="Remove all", disabled=True, layout=widgets.Layout(width="98%"))
vbox = widgets.VBox(children=[slider, label, radio, text_field, segment_button, remove_button])

# create output widget and place both columns in the top level container
out1 = widgets.Output()
hbox = widgets.HBox(children=[vbox, out1])
display(hbox)

# add behaviours to buttons
segment_button.on_click(perform_segmentation)
remove_button.on_click(clear_all)

# Define what should be shown in the Output widget
with out1:
    fig, axes = plt.subplots()
    axes.get_xaxis().set_visible(False)
    axes.get_yaxis().set_visible(False)
    draw(axes)
    fig.canvas.mpl_connect("button_press_event", on_canvas_click)
    fig.canvas.draw()

FileNotFoundError: No such file: '/Users/dalonsoa/Documents/Projects/PYTHON-GUIS/python-guis/python_guis/jupyter_widgets/insects.jpg'